In [1]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from transformers import Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [4]:
# load dataset
data = load_dataset("json", data_files="/content/programming_jokes.json")

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
# load the model and tokenizer
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [6]:
# ensure the tokenizer has a padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [7]:
# tokenize the dataset
def tokenize_function(examples):
    inputs =  tokenizer(examples["text"], padding="max_length", truncation=True, max_length=50)
    inputs["labels"] = inputs["input_ids"].copy()
    return inputs

tokenized_data = data.map(tokenize_function, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [8]:
# prepare tbe model for LoRA tuning
# LoRA configurations
lora_config = LoraConfig(
    task_type = "CAUSAL_LM",
    inference_mode = False,
    r = 8,
    lora_alpha = 16,
    lora_dropout = 0.1
)

In [9]:
# prepare the model for lora
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [17]:
# define training arguments
training_args = TrainingArguments(
    output_dir = "./results",
    overwrite_output_dir = "True",
    num_train_epochs = 30,
    per_device_train_batch_size = 4,
    save_steps = 100,
    save_total_limit = 2,
    learning_rate = 5e-5,
    logging_dir = "./logs",
    logging_steps = 10,
    fp16 = True,
    report_to = "none" # Disable wandb reporting explicitly
)

In [18]:
# define the trainer
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_data["train"],
    tokenizer = tokenizer
)

<ipython-input-18-4fed5b5ab6d7>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [19]:
# train the model
trainer.train()

Step,Training Loss
10,8.151500
20,8.118600
30,7.875500
40,7.360300
50,6.903400
60,6.547400
70,6.049700
80,5.137400
90,4.761200
100,4.238700


TrainOutput(global_step=7500, training_loss=0.3464901394685109, metrics={'train_runtime': 247.1987, 'train_samples_per_second': 121.36, 'train_steps_per_second': 30.34, 'total_flos': 384086016000000.0, 'train_loss': 0.3464901394685109, 'epoch': 30.0})

In [20]:
model.save_pretrained('./fine_tuned_model')
tokenizer.save_pretrained('./fine_tuned_model')

('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.json',
 './fine_tuned_model/merges.txt',
 './fine_tuned_model/added_tokens.json',
 './fine_tuned_model/tokenizer.json')

In [21]:
!zip -r /content/fine_tuned_model.zip /content/fine_tuned_model

  adding: content/fine_tuned_model/ (stored 0%)
  adding: content/fine_tuned_model/merges.txt (deflated 53%)
  adding: content/fine_tuned_model/vocab.json (deflated 59%)
  adding: content/fine_tuned_model/special_tokens_map.json (deflated 60%)
  adding: content/fine_tuned_model/adapter_model.safetensors (deflated 7%)
  adding: content/fine_tuned_model/tokenizer_config.json (deflated 54%)
  adding: content/fine_tuned_model/tokenizer.json (deflated 82%)
  adding: content/fine_tuned_model/adapter_config.json (deflated 54%)
  adding: content/fine_tuned_model/README.md (deflated 66%)


In [22]:
import zipfile
import os

# Path to the .zip file
zip_path = "/content/fine_tuned_model.zip"
extracted_path = "/content/fine_tuned_model"

# Extract the .zip file
try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_path)
    print(f"Model extracted to {extracted_path}")
except Exception as e:
    print(f"Error extracting the zip file: {e}")

# Load the model and tokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM

try:
    tokenizer = AutoTokenizer.from_pretrained(extracted_path)
    model = AutoModelForCausalLM.from_pretrained(extracted_path)
    print("Model loaded successfully")
except Exception as e:
    print(f"An error occurred while loading the model: {e}")

Model extracted to /content/fine_tuned_model
Model loaded successfully


In [31]:
# input prompt
try:
  prompt = input('Enter your prompt: ')
  print('Prompt entered successfully')
except Exception as e:
    print(f"An error occurred: {e}")

Enter your prompt: Hello
Prompt entered successfully


In [32]:
# tokenize the input
try:
  input_ids = tokenizer.encode(prompt, return_tensors='pt')
  print('Input tokenized successfully')
except Exception as e:
    print(f"An error occurred: {e}")

Input tokenized successfully


In [33]:
outputs = model.generate(
    input_ids,
    max_length=50,
    num_return_sequences=1,
    temperature=0.7,
    top_p=0.9,
    top_k=50,
    do_sample=True
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [34]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Hello I did not get any commands.
